In [1]:
import numpy as np
import pandas as pd

In [18]:
# 산사태 취약지역 데이터 불러오기
df_land = pd.read_csv('data/raw/landslide.csv', encoding='cp949')

In [22]:
# 충남 지역 및 해소되지 않은 취약지역만 선별
df_land_chgnam = df_land.loc[df_land['시도 (*)'] == '충청남도', :]
df_land_chgnam = df_land_chgnam.loc[df_land['취약지역 \n현재현황'].apply(lambda x: '해소' not in str(x))]
df_land_chgnam['법정동명'] = df_land_chgnam['시도 (*)'] + ' ' + df_land_chgnam['시군구 (*)'] + ' ' + df_land_chgnam['읍면동 (*)']
df_land_chgnam = df_land_chgnam.loc[:, ['법정동명', '취약지역유형', '지정일']]
df_land_chgnam.reset_index(drop=True, inplace=True)
print(len(df_land_chgnam))
df_land_chgnam.head(3)

245


,법정동명,취약지역유형,지정일
0,충청남도 보령시 청라면,토석류,2015-09-08
1,충청남도 공주시 정안면,토석류,2015-06-30
2,충청남도 공주시 쌍신동,산사태,2021-02-04


In [24]:
# 법정동코드 데이터 불러오기
law_code = pd.read_table("data/법정동코드 전체자료.txt", sep='\t', encoding='cp949')
# 읍면동 단위로 정제
law_code = law_code[law_code['법정동코드'].apply(lambda x: str(x).startswith('44'))]
law_code = law_code[law_code['폐지여부'] == '존재']
law_code = law_code[law_code['법정동명'].apply(lambda x: x[-1] in ('읍', '면', '동'))]
law_code.loc[:, '법정동코드'] = law_code.loc[:, '법정동코드'].apply(lambda x: str(x)[:-2])
law_code.reset_index(drop=True, inplace=True)
law_code.drop(columns='폐지여부', inplace=True)
print(len(law_code))
law_code.head(3)

285


,법정동코드,법정동명
0,44131101,충청남도 천안시 동남구 대흥동
1,44131102,충청남도 천안시 동남구 성황동
2,44131103,충청남도 천안시 동남구 문화동


In [27]:
# 법정동명으로 merge
df_land_pp = pd.merge(df_land_chgnam, law_code, how='left', on='법정동명')
df_land_pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동명    245 non-null    object
 1   취약지역유형  245 non-null    object
 2   지정일     245 non-null    object
 3   법정동코드   245 non-null    object
dtypes: object(4)
memory usage: 7.8+ KB


In [28]:
# 처리 데이터 저장
df_land_pp.to_csv("data/landslide_pp.csv", index=False)